In [1]:
#!pip install huggingface_hub transformers torch accelerate datasets peft nervaluate

In [2]:
#!pip install -U bitsandbytes

In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "meta-llama/Llama-2-7b-hf"
llama2_7b_t = AutoTokenizer.from_pretrained(model_name)

KeyboardInterrupt: 

In [ ]:
# pad token definitions for both tokenizer and model allow for batch size > 1
llama2_7b_t.pad_token = llama2_7b_t.eos_token
#model.config.pad_token_id = model.config.eos_token_id

In [ ]:
import pandas as pd
from datasets import load_dataset

ds = load_dataset("tner/tweetner7")

README.md:   0%|          | 0.00/27.4k [00:00<?, ?B/s]

tweetner7.py:   0%|          | 0.00/3.56k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/447k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/96.9k [00:00<?, ?B/s]

...

Generating train_all split:   0%|          | 0/7111 [00:00<?, ? examples/s]

Generating validation_random split:   0%|          | 0/576 [00:00<?, ? examples/s]

Generating train_random split:   0%|          | 0/4616 [00:00<?, ? examples/s]

Generating extra_2020 split:   0%|          | 0/87880 [00:00<?, ? examples/s]

Generating extra_2021 split:   0%|          | 0/93594 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

tweetner7_train = ds["train_all"].to_pandas()
tweetner7_test = ds["test_2021"].to_pandas()

tweetner7_big = pd.concat([tweetner7_train, tweetner7_test])
len(tweetner7_big)

9918

In [ ]:
tweetner7_validation20 = ds["validation_2020"].to_pandas()
tweetner7_validation21 = ds["validation_2021"].to_pandas()
tweetner7_test20 = ds["test_2020"].to_pandas()

tweetner7_bigger = pd.concat([tweetner7_big, tweetner7_validation20, tweetner7_validation21, tweetner7_test20])
len(tweetner7_bigger)

11380

In [ ]:
entity_dict_tweetner = {
    0: "B-corporation",
    1: "B-creative_work",
    2: "B-event",
    3: "B-group",
    4: "B-location",
    5: "B-person",
    6: "B-product",
    7: "I-corporation",
    8: "I-creative_work",
    9: "I-event",
    10: "I-group",
    11: "I-location",
    12: "I-person",
    13: "I-product",
    14: "O"
}

def tags_to_labels(col):
    result = []
    for i in col:
        label = entity_dict_tweetner[i]
        result.append(label)

    return result

# fine-tuning with tweetner7

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(tweetner7_bigger)

train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
label_list = list(entity_dict_tweetner.values())
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = llama2_7b_t(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_id = None
        for word_id in word_ids:
            if word_id is None:  # Special tokens
                label_ids.append(-100)
            elif word_id != previous_word_id:  # First token of a word
                label_ids.append(label[word_id])
            else:  # Subword tokens
                label_ids.append(-100)
            previous_word_id = word_id
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/9104 [00:00<?, ? examples/s]

Map:   0%|          | 0/2276 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,  # rank of the low-rank adaptation
    lora_alpha=32,  # scaling factor
    target_modules=["q_proj", "v_proj"],  # target modules for LoRA
    lora_dropout=0.1,  # dropout rate
    bias="none",  # bias handling
    task_type="TOKEN_CLS" # NER is a form of token classification
)

model = get_peft_model(model, lora_config)

In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import classification_report

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # flattening predictions and labels
    true_labels = [id_to_label[label] for label in labels.flatten() if label != -100]
    true_predictions = [id_to_label[pred] for (pred, label) in zip(predictions.flatten(), labels.flatten()) if label != -100]

    # calculating metrics
    report = classification_report(true_labels, true_predictions, output_dict=True)
    return {
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1": report["weighted avg"]["f1-score"]
    }

training_args = TrainingArguments(
    output_dir="./ner-llama-2-7b",
    evaluation_strategy="epoch",   # evaluating at every epoch
    learning_rate=5e-5,
    per_device_train_batch_size=128, # adjust batch size depending on resources,
    per_device_eval_batch_size=128,  # high batch size because NER needs less RAM than SA
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500, # higher number to save time
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=llama2_7b_t,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ðŸ¤— Transformers. Use `eval_strategy` instead
  warnings.warn(


NameError: name 'model' is not defined

In [ ]:
# with 2 epochs, tweetner7-big
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: hausbichler-georg (hausbichler-georg-wirtschaftsuniversit-t-wien) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.681055,0.775410,0.851387,0.806359
2,No log,0.553897,0.802113,0.863784,0.818706


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=124, training_loss=1.1178052348475302, metrics={'train_runtime': 4850.4284, 'train_samples_per_second': 3.271, 'train_steps_per_second': 0.026, 'total_flos': 7.897649936980992e+16, 'train_loss': 1.1178052348475302, 'epoch': 2.0})

In [ ]:
model.save_pretrained("ner-llama-2-7b-finetuned-tweetner7")
llama2_7b_t.save_pretrained("ner-llama-2-7b-finetuned-tweetner7")

('ner-llama-2-7b-finetuned-tweetner7/tokenizer_config.json',
 'ner-llama-2-7b-finetuned-tweetner7/special_tokens_map.json',
 'ner-llama-2-7b-finetuned-tweetner7/tokenizer.model',
 'ner-llama-2-7b-finetuned-tweetner7/added_tokens.json',
 'ner-llama-2-7b-finetuned-tweetner7/tokenizer.json')

In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.5538966059684753,
 'eval_precision': 0.8021131429416545,
 'eval_recall': 0.8637838011432976,
 'eval_f1': 0.8187060127413887,
 'eval_runtime': 185.1272,
 'eval_samples_per_second': 10.717,
 'eval_steps_per_second': 0.086,
 'epoch': 2.0}

In [ ]:
# with 3 epochs, tweetner7-big
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 Â·Â·Â·Â·Â·Â·Â·Â·Â·Â·


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hausbichler-georg (hausbichler-georg-wirtschaftsuniversit-t-wien) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.747603,0.783047,0.843396,0.809564
2,No log,0.510679,0.821797,0.869119,0.837187
3,No log,0.454969,0.839352,0.878753,0.850002


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=186, training_loss=0.8950547249086441, metrics={'train_runtime': 7233.6397, 'train_samples_per_second': 3.29, 'train_steps_per_second': 0.026, 'total_flos': 1.1846474905471488e+17, 'train_loss': 0.8950547249086441, 'epoch': 3.0})

In [ ]:
model.save_pretrained("ner-llama-2-7b-finetuned-tweetner7-3")
llama2_7b_t.save_pretrained("ner-llama-2-7b-finetuned-tweetner7-3")

('ner-llama-2-7b-finetuned-tweetner7-3/tokenizer_config.json',
 'ner-llama-2-7b-finetuned-tweetner7-3/special_tokens_map.json',
 'ner-llama-2-7b-finetuned-tweetner7-3/tokenizer.model',
 'ner-llama-2-7b-finetuned-tweetner7-3/added_tokens.json',
 'ner-llama-2-7b-finetuned-tweetner7-3/tokenizer.json')

In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.45496857166290283,
 'eval_precision': 0.8393520916305982,
 'eval_recall': 0.8787525363071368,
 'eval_f1': 0.8500017982711692,
 'eval_runtime': 183.8459,
 'eval_samples_per_second': 10.792,
 'eval_steps_per_second': 0.087,
 'epoch': 3.0}

In [ ]:
# with 4 epochs, tweetner7bigger
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 Â·Â·Â·Â·Â·Â·Â·Â·Â·Â·


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hausbichler-georg (hausbichler-georg-wirtschaftsuniversit-t-wien) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.608548,0.781358,0.857285,0.810503
2,No log,0.448151,0.850950,0.882938,0.853018
3,No log,0.397302,0.870662,0.893005,0.870250
4,No log,0.382102,0.876893,0.896532,0.879017


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=288, training_loss=0.754784795973036, metrics={'train_runtime': 11062.1204, 'train_samples_per_second': 3.292, 'train_steps_per_second': 0.026, 'total_flos': 1.8124579033595904e+17, 'train_loss': 0.754784795973036, 'epoch': 4.0})

In [ ]:
model.save_pretrained("ner-llama-2-7b-finetuned-tweetner7-4")
llama2_7b_t.save_pretrained("ner-llama-2-7b-finetuned-tweetner7-4")

('ner-llama-2-7b-finetuned-tweetner7-4/tokenizer_config.json',
 'ner-llama-2-7b-finetuned-tweetner7-4/special_tokens_map.json',
 'ner-llama-2-7b-finetuned-tweetner7-4/tokenizer.model',
 'ner-llama-2-7b-finetuned-tweetner7-4/added_tokens.json',
 'ner-llama-2-7b-finetuned-tweetner7-4/tokenizer.json')

In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.38210174441337585,
 'eval_precision': 0.876892813143281,
 'eval_recall': 0.896531695984069,
 'eval_f1': 0.8790165691166707,
 'eval_runtime': 211.1368,
 'eval_samples_per_second': 10.78,
 'eval_steps_per_second': 0.085,
 'epoch': 4.0}

## applying model

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import BitsAndBytesConfig
from peft import PeftModel

# first loading base model, then reloading fine-tuned model on top of it with PEFT

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForTokenClassification.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)

model_ft = PeftModel.from_pretrained(
    base_model,
    "ner-llama-2-7b-finetuned-tweetner7-4"
)

tokenizer_ft = AutoTokenizer.from_pretrained("ner-llama-2-7b-finetuned-tweetner7-4")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
test_df = pd.DataFrame(test_dataset)
test_df.head()

,tokens,tags,id,date,__index_level_0__
0,"[Rapper, Kanye, West, to, hold, first, preside...","[14, 5, 12, 14, 14, 14, 2, 9, 14, 14, 14, 6]",1284876593433845761,2020-07-19,900
1,"[Morrison, uses, UN, speech, to, slam, Austral...","[5, 14, 14, 14, 14, 14, 4, 14, 14, 14, 14, 14,...",1177038714398052354,2019-09-26,258
2,"[Hi, Chan,If, you, saw, this, tweet, I, want, ...","[14, 5, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14...",1185886306322112512,2019-10-20,1380
3,"[DeAngelo, Hall, says, free, agent, Chris, Har...","[5, 12, 14, 14, 14, 5, 12, 14, 14, 14, 14, 14,...",1227045476190560256,2020-02-11,39
4,"[The, greatest, thing, in, the, world, is, to,...","[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1...",1398848827239501825,2021-05-30,1918


In [16]:
def llama_pred(model, tokenizer, df, text_col, tag_col=None, max_length=128):
    # setting padding tokens
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

    # moving model to GPU and setting it to evaluation-mode
    model.to(model.device)
    model.eval()

    batch_size = 32
    num_batches = (len(df) + batch_size - 1) // batch_size
    all_predictions = []

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(df))
        if start_idx >= len(df):
            break

        batch_tokens = df[text_col][start_idx:end_idx].tolist()

        # tokenizing with the same parameters as during training
        batch_inputs = tokenizer(
            batch_tokens,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            is_split_into_words=True,  # very important for token classification
            return_tensors="pt"
        )

        batch_inputs = {key: value.to(model.device) for key, value in batch_inputs.items()}

        with torch.no_grad():
            outputs = model(**batch_inputs)
            batch_predictions = torch.argmax(outputs.logits, dim=-1)

            processed_predictions = []
            for j, preds in enumerate(batch_predictions):
                word_ids = tokenizer(batch_tokens[j], is_split_into_words=True).word_ids()
                previous_word_id = None
                word_preds = []

                for word_id, pred in zip(word_ids, preds):
                    if word_id is None:  # handling special tokens
                        continue
                    elif word_id != previous_word_id:  # first token of a word
                        word_preds.append(pred.item())
                    previous_word_id = word_id

                processed_predictions.append(word_preds)

            all_predictions.extend(processed_predictions)

    df["llama_ft_prediction"] = all_predictions
    return df

In [ ]:
test_df = llama_pred(model_ft, tokenizer_ft, test_df, "tokens")

In [ ]:
test_df["true_labels"] = test_df["tags"].apply(tags_to_labels)
test_df["pred_labels"] = test_df["llama_ft_prediction"].apply(tags_to_labels)
test_df.head()

,tokens,tags,id,date,__index_level_0__,llama_ft_prediction,true_labels,pred_labels
0,"[Rapper, Kanye, West, to, hold, first, preside...","[14, 5, 12, 14, 14, 14, 2, 9, 14, 14, 14, 6]",1284876593433845761,2020-07-19,900,"[14, 5, 12, 14, 14, 14, 14, 14, 14, 14, 14, 6]","[O, B-person, I-person, O, O, O, B-event, I-ev...","[O, B-person, I-person, O, O, O, O, O, O, O, O..."
1,"[Morrison, uses, UN, speech, to, slam, Austral...","[5, 14, 14, 14, 14, 14, 4, 14, 14, 14, 14, 14,...",1177038714398052354,2019-09-26,258,"[14, 14, 14, 14, 14, 14, 4, 14, 14, 14, 14, 14...","[B-person, O, O, O, O, O, B-location, O, O, O,...","[O, O, O, O, O, O, B-location, O, O, O, O, O, ..."
2,"[Hi, Chan,If, you, saw, this, tweet, I, want, ...","[14, 5, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14...",1185886306322112512,2019-10-20,1380,"[14, 5, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14...","[O, B-person, O, O, O, O, O, O, O, O, O, O, B-...","[O, B-person, O, O, O, O, O, O, O, O, O, O, B-..."
3,"[DeAngelo, Hall, says, free, agent, Chris, Har...","[5, 12, 14, 14, 14, 5, 12, 14, 14, 14, 14, 14,...",1227045476190560256,2020-02-11,39,"[14, 12, 14, 14, 14, 5, 12, 14, 14, 3, 14, 14,...","[B-person, I-person, O, O, O, B-person, I-pers...","[O, I-person, O, O, O, B-person, I-person, O, ..."
4,"[The, greatest, thing, in, the, world, is, to,...","[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1...",1398848827239501825,2021-05-30,1918,"[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
from nervaluate import Evaluator

true = test_df["true_labels"].values.tolist()
pred = test_df["pred_labels"].values.tolist()

evaluator = Evaluator(true, pred, tags=["corporation", "creative_work", "event", "group", "location", "person", "product"], loader="list")

results, results_by_tag, result_indices, result_indices_by_tag = evaluator.evaluate()
print(results)

{'ent_type': {'correct': 3035, 'incorrect': 1321, 'partial': 0, 'missed': 1905, 'spurious': 813, 'possible': 6261, 'actual': 5169, 'precision': 0.5871541884310312, 'recall': 0.4847468455518288, 'f1': 0.531058617672791}, 'partial': {'correct': 3434, 'incorrect': 0, 'partial': 922, 'missed': 1905, 'spurious': 813, 'possible': 6261, 'actual': 5169, 'precision': 0.7535306635712904, 'recall': 0.6221050950327424, 'f1': 0.6815398075240594}, 'strict': {'correct': 2447, 'incorrect': 1909, 'partial': 0, 'missed': 1905, 'spurious': 813, 'possible': 6261, 'actual': 5169, 'precision': 0.473399110079319, 'recall': 0.39083213544162276, 'f1': 0.42817147856517934}, 'exact': {'correct': 3434, 'incorrect': 922, 'partial': 0, 'missed': 1905, 'spurious': 813, 'possible': 6261, 'actual': 5169, 'precision': 0.6643451344554072, 'recall': 0.5484746845551829, 'f1': 0.6008748906386702}}


In [35]:
from sklearn.metrics import precision_score, recall_score, classification_report

# function returns dictionary containing precision, recall and f1-score
def evaluate_ner_predictions(df, true_col="tags", pred_col="llama_ft_prediction"):
    y_true_flat = []
    y_pred_flat = []

    for true_seq, pred_seq in zip(df[true_col], df[pred_col]):
       # skipping padding/special tokens (marked as -100 in ground truth)
       for true_label, pred_label in zip(true_seq, pred_seq):
            if true_label != -100:
               y_true_flat.append(true_label)
               y_pred_flat.append(pred_label)

    precision = precision_score(y_true_flat, y_pred_flat, average='weighted')
    recall = recall_score(y_true_flat, y_pred_flat, average='weighted')
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(classification_report(y_true_flat, y_pred_flat))

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
metrics = evaluate_ner_predictions(test_df)

## predicting on new data

In [ ]:
df_tweets_bigtech = pd.read_csv("tweets_bigtech_10ksample.csv")
df_tweets_bigtech.head()

,Unnamed: 0,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1,sentiment
0,0,2020-07-12 09:24:26,AMD,25,114,AMD,United Kingdom,0.0,moffphcgaming,#AMD,Been on holiday so back now. Gonna try get som...,1.282244e+18,ðŸ•¹MoffPHC GamingðŸ•¹,-0.3102,Technology,AMD,0
1,1,2020-07-12 08:16:45,AMD,1719,1,AMD,digitalocean,1.0,LinuxDreams,#AMD,RT @LinuxReviews: #Linux architect Linus Torva...,1.282227e+18,LinuxDreams,-0.3612,Technology,AMD,0
2,2,2020-07-12 08:11:41,AMD,69,135,AMD,Amsterdam,1.0,LinuxReviews,#AMD,"#Linux architect Linus Torvalds: AVX512 Is ""A ...",1.282226e+18,LinuxReviews,-0.3612,Technology,AMD,0
3,3,2020-07-12 02:22:50,AMD,34,155,AMD,San Francisco,0.0,NdrewGarcia,#AMD,#AMD stuck in a range box chart https://t.co/5...,1.282138e+18,Encino_Man,-0.2500,Technology,AMD,0
4,4,2020-07-11 23:58:44,AMD,802,730,AMD,"New Jersey, USA",0.0,Roger_Clinton1,#AMD,$AMD Epyc Milan Leak â€“ Three early Genesis sam...,1.282102e+18,Roger Ocasio-Clinton,-0.3400,Technology,AMD,0


In [ ]:
df_tweets_bigtech["tokens"] = df_tweets_bigtech["text"].apply(lambda x: x.split())
df_tweets_bigtech = llama_pred(model_ft, tokenizer_ft, df_tweets_bigtech, "tokens")

In [ ]:
df_tweets_bigtech["pred_labels"] = df_tweets_bigtech["llama_ft_prediction"].apply(tags_to_labels)
df_tweets_bigtech.head()

,Unnamed: 0,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1,sentiment,tokens,llama_ft_prediction,pred_labels
0,0,2020-07-12 09:24:26,AMD,25,114,AMD,United Kingdom,0.0,moffphcgaming,#AMD,Been on holiday so back now. Gonna try get som...,1.282244e+18,ðŸ•¹MoffPHC GamingðŸ•¹,-0.3102,Technology,AMD,0,"[Been, on, holiday, so, back, now., Gonna, try...","[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, I-person,..."
1,1,2020-07-12 08:16:45,AMD,1719,1,AMD,digitalocean,1.0,LinuxDreams,#AMD,RT @LinuxReviews: #Linux architect Linus Torva...,1.282227e+18,LinuxDreams,-0.3612,Technology,AMD,0,"[RT, @LinuxReviews:, #Linux, architect, Linus,...","[14, 14, 14, 14, 5, 12, 14, 14, 14, 14, 14, 14...","[O, O, O, O, B-person, I-person, O, O, O, O, O..."
2,2,2020-07-12 08:11:41,AMD,69,135,AMD,Amsterdam,1.0,LinuxReviews,#AMD,"#Linux architect Linus Torvalds: AVX512 Is ""A ...",1.282226e+18,LinuxReviews,-0.3612,Technology,AMD,0,"[#Linux, architect, Linus, Torvalds:, AVX512, ...","[14, 14, 5, 12, 14, 14, 14, 14, 14, 14, 14, 14...","[O, O, B-person, I-person, O, O, O, O, O, O, O..."
3,3,2020-07-12 02:22:50,AMD,34,155,AMD,San Francisco,0.0,NdrewGarcia,#AMD,#AMD stuck in a range box chart https://t.co/5...,1.282138e+18,Encino_Man,-0.2500,Technology,AMD,0,"[#AMD, stuck, in, a, range, box, chart, https:...","[14, 14, 14, 14, 14, 14, 14, 14, 14, 14]","[O, O, O, O, O, O, O, O, O, O]"
4,4,2020-07-11 23:58:44,AMD,802,730,AMD,"New Jersey, USA",0.0,Roger_Clinton1,#AMD,$AMD Epyc Milan Leak â€“ Three early Genesis sam...,1.282102e+18,Roger Ocasio-Clinton,-0.3400,Technology,AMD,0,"[$AMD, Epyc, Milan, Leak, â€“, Three, early, Gen...","[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
df_tweets_bigtech.to_csv("10k_tweets_bigtech_after_llama_ner.csv")

# fine tuning with corpus

In [4]:
import pandas as pd

df_corpus = pd.read_json("NER_corpus.json", orient="records")
df_corpus = df_corpus.rename(columns={"labels":"original_labels"})
df_corpus.head()

,tokens,original_labels,tags
0,"[., {{MENTION}}, stand, tall, in, the, red, zo...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1..."
1,"[Who, else, is, gonna, get, the, new, Surface,...","[O, O, O, O, O, O, O, B-product, I-product, O,...","[10, 10, 10, 10, 10, 10, 10, 4, 9, 10, 10, 10]"
2,"[EDIE, IN, BETWEEN, comes, out, THIS, MONTH, ,...","[B-person, O, O, O, O, O, O, O, O, O, O, O, O,...","[3, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10..."
3,"[â€˜, Joe, Biden, may, have, won, the, keys, to,...","[O, B-person, I-person, O, O, O, O, O, O, O, B...","[10, 3, 8, 10, 10, 10, 10, 10, 10, 10, 2, 7, 1..."
4,"[Imma, hit, reload, until, I, see, additional,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1..."


In [5]:
entity_dict = {
    0: "B-corporation",
    1: "B-event",
    2: "B-location",
    3: "B-person",
    4: "B-product",
    5: "I-corporation",
    6: "I-event",
    7: "I-location",
    8: "I-person",
    9: "I-product",
    10: "O"
}

label_list = list(entity_dict.values())
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

In [6]:
# examining length to determine maxlength
df_corpus["tokens"].str.len().agg(['mean','max','std'])

,tokens
mean,25.203704
max,133.000000
std,13.413614


In [ ]:
len(df_corpus[df_corpus["tokens"].apply(len)>=128])

2

In [6]:
from transformers import AutoTokenizer
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "meta-llama/Llama-2-7b-hf"
llama_t = AutoTokenizer.from_pretrained(model_name)

llama_t.pad_token = llama_t.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
from transformers import AutoModelForTokenClassification

llama = AutoModelForTokenClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)

llama.config.pad_token_id = llama.config.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

llama = prepare_model_for_kbit_training(llama)

lora_config = LoraConfig(
    r=16,  # rank of the low-rank adaptation
    lora_alpha=32,  # scaling factor
    target_modules=["k_proj", "q_proj", "v_proj", "o_proj"],  # target modules for LoRA
    lora_dropout=0.05,  # dropout rate
    bias="none",  # bias handling
    task_type="TOKEN_CLS" # NER is a form of token classification
)

llama = get_peft_model(llama, lora_config)

In [9]:
from datasets import Dataset

def tokenize_and_align_labels(examples):
    tokenized_inputs = llama_t(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_id = None
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != previous_word_id:
                label_ids.append(label[word_id])
            else:
                label_ids.append(-100)
            previous_word_id = word_id
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

dataset = Dataset.from_pandas(df_corpus)
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/24835 [00:00<?, ? examples/s]

In [10]:
train_test_split1 = tokenized_datasets.train_test_split(test_size=0.15, seed=42)
train_val_dataset = train_test_split1['train']
test_dataset = train_test_split1['test']

train_test_split2 = train_val_dataset.train_test_split(test_size=0.15, seed=42)
train_dataset = train_test_split2['train']
val_dataset = train_test_split2['test']

In [11]:
print(id_to_label)
label_names = list(entity_dict.values())
print(label_names)

{0: 'B-corporation', 1: 'B-event', 2: 'B-location', 3: 'B-person', 4: 'B-product', 5: 'I-corporation', 6: 'I-event', 7: 'I-location', 8: 'I-person', 9: 'I-product', 10: 'O'}
['B-corporation', 'B-event', 'B-location', 'B-person', 'B-product', 'I-corporation', 'I-event', 'I-location', 'I-person', 'I-product', 'O']


In [13]:
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import classification_report

def compute_metrics(p):
     predictions, labels = p
     predictions = np.argmax(predictions, axis=2)

     # flattening predictions and labels
     true_labels = [id_to_label[label] for label in labels.flatten() if label != -100]
     true_predictions = [id_to_label[pred] for (pred, label) in zip(predictions.flatten(), labels.flatten()) if label != -100]

     # calculating metrics
     report = classification_report(true_labels, true_predictions, output_dict=True)
     return {
         "precision": report["weighted avg"]["precision"],
         "recall": report["weighted avg"]["recall"],
         "f1": report["weighted avg"]["f1-score"]
     }

training_args = TrainingArguments(
    output_dir="./ner-llama-2-7b",
    evaluation_strategy="epoch", # evaluating at every epoch
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=32, # adjust batch size depending on resources
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    label_names=label_names
)

trainer = Trainer(
    model=llama,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=llama_t,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ðŸ¤— Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-303f92546893>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

{'eval_runtime': 59.2067,
 'eval_samples_per_second': 62.932,
 'eval_steps_per_second': 1.976,
 'epoch': 1.9910873440285206}

In [ ]:
trainer.save_model("./llama2-7b-after-ner-corpus")

In [12]:
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
import numpy as np
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from tqdm import tqdm

training_args = TrainingArguments(
    output_dir="./ner-llama-2-7b",
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    num_train_epochs=7,
    save_total_limit=7,
    label_names=label_names
)

trainer = Trainer(
    model=llama,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=llama_t
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ðŸ¤— Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-557b763ed6d8>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 Â·Â·Â·Â·Â·Â·Â·Â·Â·Â·


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hausbichler-georg (hausbichler-georg-wirtschaftsuniversit-t-wien) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.213300,No log
2,0.198700,No log
3,0.180400,No log
4,0.147600,No log
5,0.154800,No log
6,0.126300,No log


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=980, training_loss=0.21192899845084365, metrics={'train_runtime': 6710.3345, 'train_samples_per_second': 18.717, 'train_steps_per_second': 0.146, 'total_flos': 6.22363941513726e+17, 'train_loss': 0.21192899845084365, 'epoch': 6.955436720142602})

In [13]:
from transformers import BitsAndBytesConfig
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_llama = AutoModelForTokenClassification.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from sklearn.metrics import classification_report

best_f1 = 0
best_epoch = 0

def calculate_metrics(df, true_label_col, pred_label_col):
    all_true = []
    all_pred = []

    for true, pred in zip(df[true_label_col], df[pred_label_col]):
        all_true.extend(true)
        all_pred.extend(pred[:len(true)])


    report = classification_report(all_true, all_pred, output_dict=True)

    return {
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1": report["weighted avg"]["f1-score"]
    }

val_df = (val_dataset).to_pandas()

folders = [141, 282, 423, 564, 705, 846, 980]
for epoch in range(7):
    foldernr = folders[epoch]
    checkpoint_dir = f"./ner-llama-2-7b/checkpoint-{str(foldernr)}"
    print(f"Loading model from {checkpoint_dir}")

    model = PeftModel.from_pretrained(
    base_llama,
    checkpoint_dir)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

    val_df_copy = val_df.copy()
    val_df_with_preds = llama_pred(
        model,
        tokenizer,
        val_df_copy,
        text_col="tokens",
        max_length=128
    )

    metrics = calculate_metrics(
        val_df_with_preds,
        true_label_col="tags",
        pred_label_col="llama_ft_prediction"
    )

    f1 = metrics["f1"]
    precision = metrics["precision"]
    recall = metrics["recall"]

    print(f"Epoch {epoch+1}: Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_epoch = epoch

print(f"Best epoch according to F1-score: {best_epoch+1}, F1: {best_f1:.4f}")

Loading model from ./ner-llama-2-7b/checkpoint-141
Epoch 1: Precision=0.9299, Recall=0.9326, F1=0.9215
Loading model from ./ner-llama-2-7b/checkpoint-282
Epoch 2: Precision=0.9318, Recall=0.9347, F1=0.9244
Loading model from ./ner-llama-2-7b/checkpoint-423
Epoch 3: Precision=0.9322, Recall=0.9344, F1=0.9250
Loading model from ./ner-llama-2-7b/checkpoint-564
Epoch 4: Precision=0.9324, Recall=0.9354, F1=0.9258
Loading model from ./ner-llama-2-7b/checkpoint-705
Epoch 5: Precision=0.9314, Recall=0.9354, F1=0.9254
Loading model from ./ner-llama-2-7b/checkpoint-846
Epoch 6: Precision=0.9313, Recall=0.9353, F1=0.9255
Loading model from ./ner-llama-2-7b/checkpoint-980
Epoch 7: Precision=0.9301, Recall=0.9339, F1=0.9247
Best epoch according to F1-score: 4, F1: 0.9258


## applying the model

### after 2 epochs

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import BitsAndBytesConfig
from peft import PeftModel

# first loading base model, then reloading fine-tuned model on top of it with PEFT

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_llama = AutoModelForTokenClassification.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)

llama_corpus = PeftModel.from_pretrained(
    base_model,
    "./llama2-7b-after-ner-corpus"
)

llama_corpus_t = AutoTokenizer.from_pretrained("./llama2-7b-after-ner-corpus")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
corpus_test = test_dataset.to_pandas()
corpus_test.head()

,tokens,original_labels,tags,input_ids,attention_mask,labels
0,"[There, has, been, no, electricity, for, last,...","[O, O, O, O, O, O, O, O, O, O, B-location, O, ...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 2, 10...","[1, 1670, 756, 1063, 694, 12646, 537, 363, 183...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 10, 10, 10, 10, 10, -100, 10, 10, 10, -..."
1,"[2-0, to, West, Ham, at, half-time, ., Think, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1...","[1, 29871, 29906, 29899, 29900, 304, 3122, 790...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 10, -100, -100, -100, 10, 10, 10, 10, 1..."
2,"[I, do, n't, know, why, this, {{MENTION}}, tea...","[O, O, O, O, O, O, B-corporation, O, O, O, O, ...","[10, 10, 10, 10, 10, 10, 0, 10, 10, 10, 10, 10...","[1, 306, 437, 302, 29915, 29873, 1073, 2020, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 10, 10, 10, -100, -100, 10, 10, 10, 0, ..."
3,"[I, guess, it, follows, that, President, Trump...","[O, O, O, O, O, B-person, I-person, O, O, O, O...","[10, 10, 10, 10, 10, 3, 8, 10, 10, 10, 10, 10,...","[1, 306, 4140, 372, 4477, 393, 7178, 27504, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 10, 10, 10, 10, 10, 3, 8, 10, 10, -100,..."
4,"[Suddenly, getting, Classic, WoW, Naxx, tips, ...","[O, O, O, O, B-product, O, O, B-person, O, O, ...","[10, 10, 10, 10, 4, 10, 10, 3, 10, 10, 10, 10,...","[1, 8383, 1145, 368, 2805, 24300, 14962, 29956...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 10, -100, -100, 10, 10, 10, -100, 4, -1..."


In [18]:
# adapted version to handle numpy arrays, probably due to json storing
def llama_pred(model, tokenizer, df, text_col, max_length=128):
    # setting padding tokens
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

    # moving model to GPU and setting it to evaluation-mode
    model.to(model.device)
    model.eval()

    batch_size = 32
    num_batches = (len(df) + batch_size - 1) // batch_size
    all_predictions = []

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(df))
        if start_idx >= len(df):
            break

        # converting numpy-arrays to lists
        batch_tokens = [tokens.tolist() if hasattr(tokens, 'tolist') else tokens for tokens in df[text_col][start_idx:end_idx]]

        encodings = tokenizer(
            batch_tokens,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            is_split_into_words=True,
            return_tensors="pt"
        )

        batch_inputs = {key: value.to(model.device) for key, value in encodings.items()}

        with torch.no_grad():
            outputs = model(**batch_inputs)
            batch_predictions = torch.argmax(outputs.logits, dim=-1)

            processed_predictions = []
            for j, (tokens, preds) in enumerate(zip(batch_tokens, batch_predictions)):
                word_ids = encodings.word_ids(batch_index=j)
                previous_word_id = None
                word_preds = []

                word_pred_map = {}
                for word_id, pred in zip(word_ids, preds):
                  pred_id = pred.item()
                  if word_id is not None:  # Skip special tokens
                      if word_id not in word_pred_map:
                          word_pred_map[word_id] = []
                      word_pred_map[word_id].append(pred.item())

                for word_idx in sorted(word_pred_map.keys()):
                    predictions = word_pred_map[word_idx]
                    most_common = max(set(predictions), key=predictions.count)
                    word_preds.append(most_common)

                processed_predictions.append(word_preds)

            all_predictions.extend(processed_predictions)

    df["llama_ft_prediction"] = all_predictions
    return df

In [ ]:
corpus_test = llama_pred(llama_corpus, llama_corpus_t, corpus_test, "tokens")

In [ ]:
from nervaluate import Evaluator

corpus_test["true_labels"] = corpus_test["tags"].apply(lambda x: [id_to_label[i] for i in x])
corpus_test["pred_labels"] = corpus_test["llama_ft_prediction"].apply(lambda x: [id_to_label[i] for i in x])

true = corpus_test["true_labels"].values.tolist()
pred = corpus_test["pred_labels"].values.tolist()

evaluator = Evaluator(true, pred, tags=["corporation", "event", "location", "person", "product"], loader="list")

results, results_by_tag, result_indices, result_indices_by_tag = evaluator.evaluate()
print(results)

{'ent_type': {'correct': 3592, 'incorrect': 184, 'partial': 0, 'missed': 3262, 'spurious': 555, 'possible': 7038, 'actual': 4331, 'precision': 0.8293696605864697, 'recall': 0.5103722648479682, 'f1': 0.6318937461518164}, 'partial': {'correct': 3275, 'incorrect': 0, 'partial': 501, 'missed': 3262, 'spurious': 555, 'possible': 7038, 'actual': 4331, 'precision': 0.8140152389748326, 'recall': 0.5009235578289287, 'f1': 0.6201952678335826}, 'strict': {'correct': 3194, 'incorrect': 582, 'partial': 0, 'missed': 3262, 'spurious': 555, 'possible': 7038, 'actual': 4331, 'precision': 0.7374740244747171, 'recall': 0.45382210855356636, 'f1': 0.5618787932096051}, 'exact': {'correct': 3275, 'incorrect': 501, 'partial': 0, 'missed': 3262, 'spurious': 555, 'possible': 7038, 'actual': 4331, 'precision': 0.7561764026783653, 'recall': 0.46533105996021595, 'f1': 0.5761280675521154}}


In [ ]:
for ent in results_by_tag.keys():
  print(f"{ent}: {results_by_tag[ent]}")

corporation: {'ent_type': {'correct': 471, 'incorrect': 71, 'partial': 0, 'missed': 609, 'spurious': 61, 'possible': 1151, 'actual': 603, 'precision': 0.7810945273631841, 'recall': 0.4092093831450912, 'f1': 0.5370581527936146}, 'partial': {'correct': 487, 'incorrect': 0, 'partial': 55, 'missed': 609, 'spurious': 61, 'possible': 1151, 'actual': 603, 'precision': 0.8532338308457711, 'recall': 0.447002606429192, 'f1': 0.5866590649942988}, 'strict': {'correct': 448, 'incorrect': 94, 'partial': 0, 'missed': 609, 'spurious': 61, 'possible': 1151, 'actual': 603, 'precision': 0.7429519071310116, 'recall': 0.38922675933970463, 'f1': 0.5108323831242875}, 'exact': {'correct': 487, 'incorrect': 55, 'partial': 0, 'missed': 609, 'spurious': 61, 'possible': 1151, 'actual': 603, 'precision': 0.8076285240464345, 'recall': 0.42311033883579496, 'f1': 0.5553021664766249}}
event: {'ent_type': {'correct': 477, 'incorrect': 25, 'partial': 0, 'missed': 468, 'spurious': 115, 'possible': 970, 'actual': 617, 'pr

In [ ]:
metrics = evaluate_ner_predictions(corpus_test)

              precision    recall  f1-score   support

           0       0.80      0.40      0.53      1150
           1       0.84      0.39      0.53       970
           2       0.84      0.60      0.70      1089
           3       0.87      0.41      0.56      2426
           4       0.94      0.19      0.31      1402
           5       0.60      0.08      0.14       232
           6       0.75      0.49      0.60       871
           7       0.70      0.56      0.62       359
           8       0.82      0.84      0.83      1082
           9       0.57      0.68      0.62       961
          10       0.95      0.99      0.97     83255

    accuracy                           0.93     93797
   macro avg       0.79      0.51      0.58     93797
weighted avg       0.93      0.93      0.92     93797

Precision: 0.9304
Recall: 0.9342
F1 Score: 0.9323


### after new method with checkpointing

In [40]:
# checking detailed entity-level performance of model after 3, 4 and 5 epochs on test set

# first loading base model, then reloading fine-tuned model on top of it with PEFT

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_llama = AutoModelForTokenClassification.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=quantization_config,
    device_map="auto",
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)

llama_corpus_3 = PeftModel.from_pretrained(
    base_llama,
    "./ner-llama-2-7b/checkpoint-423"
)

llama_corpus_3_t = AutoTokenizer.from_pretrained("./ner-llama-2-7b/checkpoint-423")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
corpus_test3 = llama_pred(llama_corpus_3, llama_corpus_3_t, corpus_test, "tokens")

In [42]:
from nervaluate import Evaluator

corpus_test3["true_labels"] = corpus_test3["tags"].apply(lambda x: [id_to_label[i] for i in x])
corpus_test3["pred_labels"] = corpus_test3["llama_ft_prediction"].apply(lambda x: [id_to_label[i] for i in x])

true = corpus_test3["true_labels"].values.tolist()
pred = corpus_test3["pred_labels"].values.tolist()

evaluator = Evaluator(true, pred, tags=["corporation", "event", "location", "person", "product"], loader="list")

results, results_by_tag, result_indices, result_indices_by_tag = evaluator.evaluate()
print(results)

{'ent_type': {'correct': 3815, 'incorrect': 202, 'partial': 0, 'missed': 3021, 'spurious': 770, 'possible': 7038, 'actual': 4787, 'precision': 0.7969500731146856, 'recall': 0.5420574026712134, 'f1': 0.6452431289640592}, 'partial': {'correct': 3521, 'incorrect': 0, 'partial': 496, 'missed': 3021, 'spurious': 770, 'possible': 7038, 'actual': 4787, 'precision': 0.7873407144349279, 'recall': 0.5355214549587951, 'f1': 0.6374630021141648}, 'strict': {'correct': 3433, 'incorrect': 584, 'partial': 0, 'missed': 3021, 'spurious': 770, 'possible': 7038, 'actual': 4787, 'precision': 0.7171506162523501, 'recall': 0.4877806194941745, 'f1': 0.5806342494714587}, 'exact': {'correct': 3521, 'incorrect': 496, 'partial': 0, 'missed': 3021, 'spurious': 770, 'possible': 7038, 'actual': 4787, 'precision': 0.73553373720493, 'recall': 0.5002841716396703, 'f1': 0.5955179704016913}}


In [43]:
for ent in results_by_tag.keys():
  print(f"{ent}: {results_by_tag[ent]}")

corporation: {'ent_type': {'correct': 483, 'incorrect': 80, 'partial': 0, 'missed': 588, 'spurious': 49, 'possible': 1151, 'actual': 612, 'precision': 0.7892156862745098, 'recall': 0.41963509991311904, 'f1': 0.547929665343165}, 'partial': {'correct': 502, 'incorrect': 0, 'partial': 61, 'missed': 588, 'spurious': 49, 'possible': 1151, 'actual': 612, 'precision': 0.8700980392156863, 'recall': 0.4626411815812337, 'f1': 0.6040839478162223}, 'strict': {'correct': 454, 'incorrect': 109, 'partial': 0, 'missed': 588, 'spurious': 49, 'possible': 1151, 'actual': 612, 'precision': 0.7418300653594772, 'recall': 0.3944396177237185, 'f1': 0.5150311968235961}, 'exact': {'correct': 502, 'incorrect': 61, 'partial': 0, 'missed': 588, 'spurious': 49, 'possible': 1151, 'actual': 612, 'precision': 0.8202614379084967, 'recall': 0.43614248479582973, 'f1': 0.5694838343732275}}
event: {'ent_type': {'correct': 486, 'incorrect': 37, 'partial': 0, 'missed': 447, 'spurious': 152, 'possible': 970, 'actual': 675, 'p

In [44]:
llama_corpus_4 = PeftModel.from_pretrained(
    base_llama,
    "./ner-llama-2-7b/checkpoint-564"
)

llama_corpus_4_t = AutoTokenizer.from_pretrained("./ner-llama-2-7b/checkpoint-564")

In [45]:
corpus_test4 = llama_pred(llama_corpus_4, llama_corpus_4_t, corpus_test, "tokens")

In [46]:
corpus_test4["true_labels"] = corpus_test4["tags"].apply(lambda x: [id_to_label[i] for i in x])
corpus_test4["pred_labels"] = corpus_test4["llama_ft_prediction"].apply(lambda x: [id_to_label[i] for i in x])

true = corpus_test4["true_labels"].values.tolist()
pred = corpus_test4["pred_labels"].values.tolist()

evaluator = Evaluator(true, pred, tags=["corporation", "event", "location", "person", "product"], loader="list")

results, results_by_tag, result_indices, result_indices_by_tag = evaluator.evaluate()
print(results)

{'ent_type': {'correct': 3796, 'incorrect': 198, 'partial': 0, 'missed': 3044, 'spurious': 685, 'possible': 7038, 'actual': 4679, 'precision': 0.8112844624919855, 'recall': 0.539357772094345, 'f1': 0.6479474268157379}, 'partial': {'correct': 3503, 'incorrect': 0, 'partial': 491, 'missed': 3044, 'spurious': 685, 'possible': 7038, 'actual': 4679, 'precision': 0.8011327206668092, 'recall': 0.532608695652174, 'f1': 0.6398395493727064}, 'strict': {'correct': 3406, 'incorrect': 588, 'partial': 0, 'missed': 3044, 'spurious': 685, 'possible': 7038, 'actual': 4679, 'precision': 0.7279333190852746, 'recall': 0.4839443023586246, 'f1': 0.5813774857045317}, 'exact': {'correct': 3503, 'incorrect': 491, 'partial': 0, 'missed': 3044, 'spurious': 685, 'possible': 7038, 'actual': 4679, 'precision': 0.7486642444966873, 'recall': 0.49772662688263714, 'f1': 0.5979346249039856}}


In [47]:
for ent in results_by_tag.keys():
  print(f"{ent}: {results_by_tag[ent]}")

corporation: {'ent_type': {'correct': 526, 'incorrect': 67, 'partial': 0, 'missed': 558, 'spurious': 76, 'possible': 1151, 'actual': 669, 'precision': 0.7862481315396114, 'recall': 0.4569939183318853, 'f1': 0.578021978021978}, 'partial': {'correct': 534, 'incorrect': 0, 'partial': 59, 'missed': 558, 'spurious': 76, 'possible': 1151, 'actual': 669, 'precision': 0.8423019431988041, 'recall': 0.4895742832319722, 'f1': 0.6192307692307693}, 'strict': {'correct': 489, 'incorrect': 104, 'partial': 0, 'missed': 558, 'spurious': 76, 'possible': 1151, 'actual': 669, 'precision': 0.7309417040358744, 'recall': 0.42484795829713295, 'f1': 0.5373626373626373}, 'exact': {'correct': 534, 'incorrect': 59, 'partial': 0, 'missed': 558, 'spurious': 76, 'possible': 1151, 'actual': 669, 'precision': 0.7982062780269058, 'recall': 0.4639443961772372, 'f1': 0.5868131868131867}}
event: {'ent_type': {'correct': 484, 'incorrect': 42, 'partial': 0, 'missed': 444, 'spurious': 142, 'possible': 970, 'actual': 668, 'pr

In [49]:
metrics = evaluate_ner_predictions(corpus_test4)

              precision    recall  f1-score   support

           0       0.79      0.44      0.56      1150
           1       0.82      0.39      0.53       970
           2       0.85      0.60      0.71      1089
           3       0.88      0.41      0.56      2426
           4       0.93      0.22      0.36      1402
           5       0.59      0.22      0.32       232
           6       0.72      0.54      0.62       871
           7       0.67      0.57      0.62       359
           8       0.67      0.87      0.76      1082
           9       0.57      0.74      0.64       961
          10       0.95      0.99      0.97     83255

    accuracy                           0.93     93797
   macro avg       0.77      0.54      0.60     93797
weighted avg       0.93      0.93      0.92     93797

Precision: 0.9318
Recall: 0.9341
F1 Score: 0.9330


In [50]:
llama_corpus_5 = PeftModel.from_pretrained(
    base_llama,
    "./ner-llama-2-7b/checkpoint-705"
)

llama_corpus_5_t = AutoTokenizer.from_pretrained("./ner-llama-2-7b/checkpoint-705")

In [51]:
corpus_test5 = llama_pred(llama_corpus_5, llama_corpus_5_t, corpus_test, "tokens")

In [52]:
corpus_test5["true_labels"] = corpus_test5["tags"].apply(lambda x: [id_to_label[i] for i in x])
corpus_test5["pred_labels"] = corpus_test5["llama_ft_prediction"].apply(lambda x: [id_to_label[i] for i in x])

true = corpus_test5["true_labels"].values.tolist()
pred = corpus_test5["pred_labels"].values.tolist()

evaluator = Evaluator(true, pred, tags=["corporation", "event", "location", "person", "product"], loader="list")

results, results_by_tag, result_indices, result_indices_by_tag = evaluator.evaluate()
print(results)

{'ent_type': {'correct': 3707, 'incorrect': 202, 'partial': 0, 'missed': 3129, 'spurious': 641, 'possible': 7038, 'actual': 4550, 'precision': 0.8147252747252747, 'recall': 0.526712134129014, 'f1': 0.6397997928891958}, 'partial': {'correct': 3418, 'incorrect': 0, 'partial': 491, 'missed': 3129, 'spurious': 641, 'possible': 7038, 'actual': 4550, 'precision': 0.8051648351648352, 'recall': 0.5205314009661836, 'f1': 0.6322920262340352}, 'strict': {'correct': 3325, 'incorrect': 584, 'partial': 0, 'missed': 3129, 'spurious': 641, 'possible': 7038, 'actual': 4550, 'precision': 0.7307692307692307, 'recall': 0.472435350951975, 'f1': 0.5738695201933034}, 'exact': {'correct': 3418, 'incorrect': 491, 'partial': 0, 'missed': 3129, 'spurious': 641, 'possible': 7038, 'actual': 4550, 'precision': 0.7512087912087912, 'recall': 0.4856493321966468, 'f1': 0.5899206075250258}}


In [53]:
for ent in results_by_tag.keys():
  print(f"{ent}: {results_by_tag[ent]}")

corporation: {'ent_type': {'correct': 509, 'incorrect': 59, 'partial': 0, 'missed': 583, 'spurious': 74, 'possible': 1151, 'actual': 642, 'precision': 0.7928348909657321, 'recall': 0.4422241529105126, 'f1': 0.5677635248187395}, 'partial': {'correct': 512, 'incorrect': 0, 'partial': 56, 'missed': 583, 'spurious': 74, 'possible': 1151, 'actual': 642, 'precision': 0.8411214953271028, 'recall': 0.46915725456125107, 'f1': 0.6023424428332403}, 'strict': {'correct': 474, 'incorrect': 94, 'partial': 0, 'missed': 583, 'spurious': 74, 'possible': 1151, 'actual': 642, 'precision': 0.7383177570093458, 'recall': 0.4118158123370982, 'f1': 0.5287228109313998}, 'exact': {'correct': 512, 'incorrect': 56, 'partial': 0, 'missed': 583, 'spurious': 74, 'possible': 1151, 'actual': 642, 'precision': 0.7975077881619937, 'recall': 0.4448305821025195, 'f1': 0.5711098717233686}}
event: {'ent_type': {'correct': 453, 'incorrect': 43, 'partial': 0, 'missed': 474, 'spurious': 99, 'possible': 970, 'actual': 595, 'pre